In [3]:
import sys
import os
import numpy as np
from scipy.optimize import curve_fit
import random
import matplotlib
import matplotlib.pyplot as plt
import datetime
import pdb #python debugger
from timeit import default_timer as timer #timer
from collections import OrderedDict
import importlib
sys.path.insert(1, '/Users/ryanlopez/ABPTutorial/c++') #Connects to ABP Folder github.com/ryanlopezzzz/ABPTutorial
from cppmd.builder import *
import cppmd as md
import read_data as rd #reads snapshot text data
import directories #used to create directories for saving data
import Physical_Quantities.vorticity as vorticity
import Physical_Quantities.fourier_calculations as fc
importlib.reload(vorticity)
importlib.reload(fc)

<module 'Physical_Quantities.fourier_calculations' from '/Users/ryanlopez/Desktop/Python_Programs/Dr_Marchetti_Research/ABP/Physical_Quantities/fourier_calculations.py'>

# Load Simulation Data

In [4]:
#Directory where all data is saved
save_dir = "/Users/ryanlopez/Desktop/Python_Programs/Dr_Marchetti_Research/Saved_Data"
exp_folder_name = "Vary_phi_and_Dr=0.2" #Folder name of experiment directory, don't change inbetween runs unless studying something different
exp_dir = os.path.join(save_dir, exp_folder_name)

varied_values = []
vicsek_values = []
velocity_values = []
dir_dot_vel_values = []
dir_dot_vel_norm_values = []
v_mag_data_values = []
exp_data = []

for f in os.scandir(exp_dir): #runs through all immediate subdirectories
    if f.is_dir() and f.path != os.path.join(exp_dir, ".ipynb_checkpoints"):
        run_desc_file = open(os.path.join(f, 'run_desc.json'))
        run_desc = json.loads(run_desc_file.read())
        run_desc_file.close()

        varied_values.append(run_desc['approx_packing'])
        vicsek_values.append(run_desc['vicsek_param'])
        velocity_values.append(run_desc['vel_param'])
        dir_dot_vel = np.load(os.path.join(f, "dir_dot_vel.npy"))
        dir_dot_vel_values.append(np.average(dir_dot_vel))
        dir_dot_vel_norm = np.load(os.path.join(f, "dir_dot_vel_norm.npy"))
        dir_dot_vel_norm_values.append(np.average(dir_dot_vel_norm))
        v_mag_data = np.load(os.path.join(f, "v_mag_data.npy"))
        v_mag_data = v_mag_data.flatten()
        v_mag_data_values.append(v_mag_data)
        exp_data.append(rd.get_exp_data(os.path.join(f, 'snapshot_data')))

In [5]:
print(varied_values)

[0.500090259142865, 0.600108310971438, 0.800144414628584, 1.100198570114303, 1.200216621942876, 1.300234673771449, 1.00018051828573, 0.900162466457157, 0.7001263628000111, 0.400072207314292]


In [6]:
view_index = 7

# Vicsek Order Parameter

In [ ]:
fig, ax = plt.subplots()
ax.scatter(varied_values, vicsek_values, label = "Vicsek Order Parameter")
ax.scatter(varied_values, dir_dot_vel_norm_values, label = "Dir dot Vel")

ax.set(xlabel=r'Packing fraction', ylabel='Value',
       title=r'$D_r = 0.2$ system')
ax.legend()

fig.text(.5, -.1, r"$D_r= 0.2$, $v_0=0.03$, J=1, k=1.", ha='center', fontsize=12)
fig.text(.5, -.2, r"Dir dot Vel = $\frac{1}{N}\sum \frac{\vec{n}_i \cdot \vec{v}_i}{\left|\vec{v}_i \right|}$, Vicsek Order Parameter = $\frac{ \left| \sum \vec{n}_i \right|}{N}$", ha='center', fontsize=12)

plt.show()

# Velocity Distribution

In [ ]:
fig, ax = plt.subplots()
ax.hist(v_mag_data_values[view_index], bins=60)

ax.set(xlabel=r'Velocity', ylabel='Num Instances',
       title=r'$D_r=0.2$ System')

fig.text(.5, -.1, r"Packing fraction $\phi= %.2f$, $v_0=0.03$, J=1, k=1, $D_r=0.2"%(varied_values[view_index]), ha='center')

plt.show()

# Vorticity

I still need to account for periodicity of box.

In [ ]:
x = np.linspace(0, 70, num=20)
y = np.linspace(0, 70, num=20)
h=2

X, Y = np.meshgrid(x,y)
vorticity_grid = vorticity.get_vorticity(exp_data[view_index], x, y, h)
vorticity_grid = np.nan_to_num(vorticity_grid)

In [ ]:
plot = plt.pcolormesh(X,Y,vorticity_grid[46])
plt.colorbar(plot)
plt.show()

# Static Structure Factor

In [ ]:
quantity = np.full(exp_data[view_index]['x'].shape, 1) #All 1 value for static structure factor
wavelength_powers, bins = fc.calculate_fourier(quantity, exp_data[view_index], L=70 , min_wave_length=1, num_bins=24)

In [ ]:
bin_centers = []
for i in range(len(bins)-1):
    bin_centers.append((bins[i]+bins[i+1]) / 2.0)
bin_centers = np.array(bin_centers)

In [ ]:
def ssf(q, nu):
    return q**(-nu)
popt, pcov = curve_fit(ssf, bin_centers, wavelength_powers)
print('nu=', popt[0])

In [ ]:
fig, ax = plt.subplots()
ax.plot(bin_centers,wavelength_powers, 'bo')
#ax.plot(bin_centers, ssf(bin_centers, *popt))
ax.set(xlabel=r'Frequency = $2 \pi$ / wavelength', ylabel='S(q)', title='Static Structure Factor')
fig.text(.5, -.1, r"$D_r= 0.2$, $v_0=0.03$, J=1, k=1,$\phi=$%.2f, $\nu=$%.2f"%(varied_values[view_index],popt[0]), ha='center', fontsize=12)
plt.show()

# Fourier Space Velocity Correlations

In [ ]:
#Can sum vx^2 and vy^2 from linearity of average
quantity_vx = exp_data[view_index]['vx']
quantity_vy = exp_data[view_index]['vy']

#Subtract average velocity in flocking
quantity_vx -=np.average(quantity_vx, axis=-1)[:,None]
quantity_vy -= np.average(quantity_vy, axis=-1)[:,None]

wavelength_powers_vx, bins = fc.calculate_fourier(quantity_vx, exp_data[view_index], L=70 , min_wave_length=5, num_bins=12)
wavelength_powers_vy, bins = fc.calculate_fourier(quantity_vy, exp_data[view_index], L=70 , min_wave_length=5, num_bins=12)

wavelength_powers = wavelength_powers_vx + wavelength_powers_vy

In [ ]:
bin_centers = []
for i in range(len(bins)-1):
    bin_centers.append((bins[i]+bins[i+1]) / 2.0)
bin_centers = np.array(bin_centers)

In [ ]:
Np = 1 #Num of particles
v0 = 0.03
def vcf(q, xi):
    return Np*v0**2/(1+(xi*q)**2)
popt, pcov = curve_fit(vcf, bin_centers, wavelength_powers)
print(r'Correlation Length \xi=', popt[0])

In [ ]:
fig, ax = plt.subplots()
ax.plot(bin_centers,wavelength_powers, 'bo')
ax.plot(bin_centers, vcf(bin_centers, *popt))
ax.set(xlabel=r'Frequency = $2 \pi$ / wavelength', ylabel=r'$\left< \tilde{v}^2 \right>$', title='Velocity Correlations')
fig.text(.5, -.1, r"$D_r= 0.2$, $v_0=0.03$, J=1, k=1, $\phi=$%.2f, $\xi=$%.2f"%(varied_values[view_index],popt[0]), ha='center', fontsize=12)
plt.show()

# Fourier Space Director Correlations

In [ ]:
#Can sum vx^2 and vy^2 from linearity of average
quantity_nx = exp_data[view_index]['nx']
wavelength_powers_nx, bins = fc.calculate_fourier(quantity_nx, exp_data[view_index], L=70 , min_wave_length=5, num_bins=12)

quantity_ny = exp_data[view_index]['ny']
wavelength_powers_ny, bins = fc.calculate_fourier(quantity_ny, exp_data[view_index], L=70 , min_wave_length=5, num_bins=12)

wavelength_powers = wavelength_powers_nx + wavelength_powers_ny

In [ ]:
bin_centers = []
for i in range(len(bins)-1):
    bin_centers.append((bins[i]+bins[i+1]) / 2.0)

In [ ]:
fig, ax = plt.subplots()
ax.plot(bin_centers,wavelength_powers, 'bo')
ax.set(xlabel=r'Frequency = $2 \pi$ / wavelength', ylabel=r'$\left< \tilde{n}^2 \right>$', title='Director Correlations')
fig.text(.5, -.1, r"$D_r= 0.2$, $v_0=0.03$, J=1, k=1,$\phi=$%.2f"%varied_values[view_index], ha='center', fontsize=12)
plt.show()